In [ ]:
import re 
from bs4 import BeautifulSoup 
import os
import nltk 
from sklearn import feature_extraction
from gensim import models
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
documentos = []
diretorio = "dados/letras-musicas/vagalume/sertanejo/"
arquivos = os.listdir(diretorio)
for arquivo in arquivos:
    caminho = diretorio + arquivo
    if os.path.isfile(caminho) and ".html" in arquivo:
        with open(caminho,"r") as f:
            html = f.read()
        html = feature_extraction.text.strip_accents_ascii(html.lower())
        html = html.replace("</br>","")
        html = html.replace("<br>"," NLINHA ")
        html = html.replace("<p>"," NLINHA ")
        soup = BeautifulSoup(html,"html.parser")
        documentos.append(soup.text.strip().lower())

In [ ]:
docs_tokenizados = []
tokenizador = nltk.TweetTokenizer()
for doc in documentos:
    tokens = tokenizador.tokenize(doc)
    docs_tokenizados.append(tokens)

In [ ]:
import pickle

with open('modelos/docs_tokenizados_vagalume.pkl', 'wb') as fp:
    pickle.dump(docs_tokenizados, fp)

In [ ]:
palavras = []
for doc in docs_tokenizados:
    palavras.extend(doc)
unicas = set(palavras)
len(unicas)

In [ ]:
# Treinando um modelo Word2vec nos documentos
w2v_model = models.Word2Vec(docs_tokenizados, size=350, window=15, min_count=0, workers=os.cpu_count(),iter=300)

In [ ]:
w2v_model.wv["amor"]

In [ ]:
# Cria um array com o vetor das duas mil primeiras palavras
vocab = list(w2v_model.wv.vocab)[:2000]
vetores_w2v = w2v_model.wv[vocab]

In [ ]:
vetores_w2v = np.asfarray(vetores_w2v)

In [ ]:
w2v_model.most_similar(positive=["amor"])

In [ ]:
# Reduzindo a dimensionalidade dos vetores das palavras
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
vetores_reduzidos = tsne.fit_transform(vetores_w2v)

In [ ]:
# Cria um dataframe com as colunas x e y. O objetivo é transformar cada representação de uma palavra em uma coordenada.
import pandas as pd
df = pd.DataFrame(vetores_reduzidos, index=vocab, columns=['x', 'y'])
df.head()

In [ ]:
# Utiliza o matplotlib com a chave de interação ligada.
%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for word, pos in df.iterrows():
    ax.annotate(word, pos) # Coloca a palavra em cada ponto

ax.scatter(df['x'], df['y']) # Cria um scatterPlot com todos os pontos x e y

In [ ]:
w2v_model.save("modelos/w2v_sertanejo_28kmusicas_tweet_tknzr_window_15_mincount_0.model")

In [ ]:
w2v_model.wv.vectors.shape